# Предварительная настройка среды разработки

## Google Collab

### Подключение сервисов Google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Setup Complete")

### Установка необходимых библиотек

## Локальная среда

### Проверка доступности GPU

Чтобы получить GPU, нажмите _Runtime_ -> _Change runtime type_, затем измените _Hardware accelerator_ с _None_ на _GPU_.

Мы можем проверить, что нам назначен GPU, и просмотреть его характеристики:

In [1]:
!nvidia-smi

Wed Jan 11 15:49:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.92.01    Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8     6W /  74W |      9MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Установка необходимых библиотек

In [12]:
!conda install -y -c conda-forge transformers
!conda install -y -c conda-forge datasets
!conda install -y -c conda-forge librosa
!conda install -c conda-forge ffmpeg
!conda install -c conda-forge huggingface_hub

!pip install --no-input evaluate
!pip install --no-input jiwer
!pip install --no-input gradio

print("Library installation complete!")

Solving environment: done

# All requested packages already installed.

Library installation complete!


### Загрузка необходимых модулей

In [16]:
from huggingface_hub import notebook_login

print("Required libraries/modules initialization complete!")

Required libraries/modules initialization complete!


### Настройка окружения

In [15]:
# Аутентификация ноутбука в HuggingFace HUB
notebook_login()

print("Environment setup completed!")

Environment setup completed!


# Полезные ссылки

- [Fine-Tune Whisper For Multilingual ASR with Transformers (Статья)](https://huggingface.co/blog/fine-tune-whisper)